# Actividad de Consolidacion 9

## 1. Carga el dataset

Iniciar Spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Migraciones").getOrCreate()

print("SparkSession iniciada correctamente.")

SparkSession iniciada correctamente.


Carga de los datos, convertir a RDD y Dataframe. Exploracion de la información

In [ ]:
csv_path = 'migraciones.csv'

df = spark.read.option("header", True).option("inferSchema", True).csv(csv_path)
print("Dataset cargado en DataFrame 'df' con {} filas".format(df.count()))

rdd = df.rdd
print("Se generó rdd a partir de DataFrame.")

display(df.limit(10).toPandas())
print('\nEsquema del DataFrame:')
df.printSchema()

print('\nEstadísticas descriptivas:')
display(df.describe().toPandas())

Dataset cargado en DataFrame 'df' con 5 filas
Se generó rdd a partir de DataFrame.


,ID,Origen,Destino,Año,Razón,PIB_Origen,PIB_Destino,Tasa_Desempleo_Origen,Tasa_Desempleo_Destino,Nivel_Educativo_Origen,Nivel_Educativo_Destino,Población_Origen,Población_Destino
0,1,México,EEUU,2015,Económica,8900,62000,5.2,3.8,8.5,12.3,125000000,331000000
1,2,Siria,Alemania,2016,Conflicto,2500,45000,15.4,4.5,6.2,14.1,18000000,83000000
2,3,Venezuela,Colombia,2017,Política,6000,15000,14.8,10.1,9.3,11.0,28000000,50000000
3,4,India,Emiratos Árabes,2018,Laboral,2200,43000,7.2,1.8,7.8,13.0,1380000000,9800000
4,5,Argentina,España,2019,Económica,10000,34000,9.5,13.2,11.5,13.8,45000000,47000000



Esquema del DataFrame:
root
 |-- ID: integer (nullable = true)
 |-- Origen: string (nullable = true)
 |-- Destino: string (nullable = true)
 |-- Año: integer (nullable = true)
 |-- Razón: string (nullable = true)
 |-- PIB_Origen: integer (nullable = true)
 |-- PIB_Destino: integer (nullable = true)
 |-- Tasa_Desempleo_Origen: double (nullable = true)
 |-- Tasa_Desempleo_Destino: double (nullable = true)
 |-- Nivel_Educativo_Origen: double (nullable = true)
 |-- Nivel_Educativo_Destino: double (nullable = true)
 |-- Población_Origen: integer (nullable = true)
 |-- Población_Destino: integer (nullable = true)


Estadísticas descriptivas:


,summary,ID,Origen,Destino,Año,Razón,PIB_Origen,PIB_Destino,Tasa_Desempleo_Origen,Tasa_Desempleo_Destino,Nivel_Educativo_Origen,Nivel_Educativo_Destino,Población_Origen,Población_Destino
0,count,5,5,5,5,5,5,5,5,5,5,5,5,5
1,mean,3.0,None,None,2017.0,None,5920.0,39800.0,10.420000000000002,6.68,8.66,12.84,3.192E8,1.0416E8
2,stddev,1.5811388300841898,None,None,1.5811388300841898,None,3573.0938974507794,17166.828478201787,4.54004405264971,4.771477758514651,1.9552493447128425,1.2461942063739502,5.944986963820863E8,1.294300119755847E8
3,min,1,Argentina,Alemania,2015,Conflicto,2200,15000,5.2,1.8,6.2,11.0,18000000,9800000
4,max,5,Venezuela,España,2019,Política,10000,62000,15.4,13.2,11.5,14.1,1380000000,331000000


## 2.Procesamiento de datos con RDDs y DataFrames

In [ ]:
# Transformaciones sobre RDDs
# Filtrar registros donde 'Origen' no sea nulo/vacío
rdd_filtrado = rdd.filter(lambda row: row['Origen'] is not None and str(row['Origen']).strip() != '')
print("Registros tras filtrado (Origen no nulo):", rdd_filtrado.count())

# Map: pares (Origen, Destino)
rdd_pares = rdd_filtrado.map(lambda row: (row['Origen'], row['Destino']))
print("Primeros 5 pares (Origen, Destino):", rdd_pares.take(5))

# flatMap: si 'Destino' contiene múltiples países separados por coma -> desglosar
rdd_destinos_flat = rdd_filtrado.flatMap(
    lambda row: [(row['Origen'], d.strip()) for d in str(row['Destino']).split(',')] if row['Destino'] else []
)
print("Destinos desglosados (flatMap) - muestra:", rdd_destinos_flat.take(20))

# flatMap: desglosar 'Razón' si tiene múltiples razones separadas por ';'
rdd_razones = rdd_filtrado.flatMap(
    lambda row: [(row['Origen'], razon.strip()) for razon in (str(row['Razón']).split(';') if row['Razón'] else [])]
)
print("Razones por origen (muestra):", rdd_razones.collect())  # dataset pequeño -> collect seguro

# Acciones sobre RDDs
print("Total registros (rdd.count()):", rdd.count())
print("Take 3 de rdd_pares:", rdd_pares.take(3))
print("Collect (rdd_destinos_flat) - completo (dataset pequeño):", rdd_destinos_flat.collect())

# Operaciones con DataFrames: filtrado, agregaciones y ordenamiento
# Filtrado de ejemplo: migraciones desde el año 2000 en adelante (ajusta el año si quieres)
df_filtrado = df.filter(df['Año'] >= 2000)
print("\nRegistros con Año >= 2000:", df_filtrado.count())

# Agregación: conteo de registros por Origen y por Destino
top_origen = df.groupBy('Origen').count().orderBy('count', ascending=False)
top_destino = df.groupBy('Destino').count().orderBy('count', ascending=False)

print("\nTop países de origen (conteo):")
display(top_origen.toPandas())

print("\nTop países de destino (conteo):")
display(top_destino.toPandas())

# Agregaciones numéricas de ejemplo por Origen (promedios y sumas útiles)
from pyspark.sql import functions as F

stats_origen = df.groupBy('Origen').agg(
    F.count('*').alias('n_registros'),
    F.avg('PIB_Origen').alias('avg_PIB_Origen'),
    F.avg('Tasa_Desempleo_Origen').alias('avg_Tasa_Desempleo_Origen'),
    F.avg('Nivel_Educativo_Origen').alias('avg_Nivel_Educativo_Origen'),
    F.sum('Población_Origen').alias('sum_Poblacion_Origen')
).orderBy('n_registros', ascending=False)

print("\nEstadísticas agregadas por Origen:")
display(stats_origen.toPandas())

# Escritura en formato Parquet
parquet_base = "migraciones_parquet"   # carpeta de salida
top_origen.write.mode("overwrite").parquet(parquet_base + "/top_origen")
top_destino.write.mode("overwrite").parquet(parquet_base + "/top_destino")
df.write.mode("overwrite").parquet(parquet_base + "/full_df")
stats_origen.write.mode("overwrite").parquet(parquet_base + "/stats_origen")

print(f"\nResultados escritos en Parquet en '{parquet_base}' (subcarpetas: top_origen, top_destino, full_df, stats_origen).")


Registros tras filtrado (Origen no nulo): 5
Primeros 5 pares (Origen, Destino): [('México', 'EEUU'), ('Siria', 'Alemania'), ('Venezuela', 'Colombia'), ('India', 'Emiratos Árabes'), ('Argentina', 'España')]
Destinos desglosados (flatMap) - muestra: [('México', 'EEUU'), ('Siria', 'Alemania'), ('Venezuela', 'Colombia'), ('India', 'Emiratos Árabes'), ('Argentina', 'España')]
Razones por origen (muestra): [('México', 'Económica'), ('Siria', 'Conflicto'), ('Venezuela', 'Política'), ('India', 'Laboral'), ('Argentina', 'Económica')]
Total registros (rdd.count()): 5
Take 3 de rdd_pares: [('México', 'EEUU'), ('Siria', 'Alemania'), ('Venezuela', 'Colombia')]
Collect (rdd_destinos_flat) - completo (dataset pequeño): [('México', 'EEUU'), ('Siria', 'Alemania'), ('Venezuela', 'Colombia'), ('India', 'Emiratos Árabes'), ('Argentina', 'España')]

Registros con Año >= 2000: 5

Top países de origen (conteo):


,Origen,count
0,Argentina,1
1,Siria,1
2,India,1
3,Venezuela,1
4,México,1



Top países de destino (conteo):


,Destino,count
0,España,1
1,Emiratos Árabes,1
2,Alemania,1
3,EEUU,1
4,Colombia,1



Estadísticas agregadas por Origen:


,Origen,n_registros,avg_PIB_Origen,avg_Tasa_Desempleo_Origen,avg_Nivel_Educativo_Origen,sum_Poblacion_Origen
0,Argentina,1,10000.0,9.5,11.5,45000000
1,Siria,1,2500.0,15.4,6.2,18000000
2,India,1,2200.0,7.2,7.8,1380000000
3,Venezuela,1,6000.0,14.8,9.3,28000000
4,México,1,8900.0,5.2,8.5,125000000



Resultados escritos en Parquet en 'migraciones_parquet' (subcarpetas: top_origen, top_destino, full_df, stats_origen).


## 3. Consultas con Spark SQL

In [ ]:
df.createOrReplaceTempView("migraciones")

# Principales países de origen
sql_origen = """
SELECT Origen, COUNT(*) AS total
FROM migraciones
GROUP BY Origen
ORDER BY total DESC
"""
top_origen_sql = spark.sql(sql_origen)
print("Top países de ORIGEN (SQL):")
display(top_origen_sql.toPandas())

# Principales países de destino
sql_destino = """
SELECT Destino, COUNT(*) AS total
FROM migraciones
GROUP BY Destino
ORDER BY total DESC
"""
top_destino_sql = spark.sql(sql_destino)
print("\nTop países de DESTINO (SQL):")
display(top_destino_sql.toPandas())

# Principales razones de migración por región (usando Origen como región)
sql_razones_region = """
SELECT Origen AS Region, `Razón`, COUNT(*) AS total
FROM migraciones
GROUP BY Origen, `Razón`
ORDER BY Region, total DESC
"""
razones_region_sql = spark.sql(sql_razones_region)
print("\nPrincipales RAZONES de migración por región (usando Origen como región):")
display(razones_region_sql.toPandas())

Top países de ORIGEN (SQL):


,Origen,total
0,Argentina,1
1,Siria,1
2,India,1
3,Venezuela,1
4,México,1



Top países de DESTINO (SQL):


,Destino,total
0,España,1
1,Emiratos Árabes,1
2,Alemania,1
3,EEUU,1
4,Colombia,1



Principales RAZONES de migración por región (usando Origen como región):


,Region,Razón,total
0,Argentina,Económica,1
1,India,Laboral,1
2,México,Económica,1
3,Siria,Conflicto,1
4,Venezuela,Política,1


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col, when

# Conversión de datos a formato MLlib
# Creamos una variable binaria de ejemplo como "etiqueta":
# Queremos predecir si la Población_Destino > Población_Origen
# 1 = mayor población en destino, 0 = caso contrario

df_ml = df.withColumn(
    "label",
    when(col("Población_Destino") > col("Población_Origen"), 1).otherwise(0)
)

# Selección de variables socioeconómicas como features
feature_cols = [
    "PIB_Origen", "PIB_Destino",
    "Tasa_Desempleo_Origen", "Tasa_Desempleo_Destino",
    "Nivel_Educativo_Origen", "Nivel_Educativo_Destino"
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data_ml = assembler.transform(df_ml).select("features", "label")

print("Dataset para MLlib listo:")
display(data_ml.toPandas())

# Modelo de Regresión Logística

lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(data_ml)   # Con 5 registros, todo se usa para entrenamiento

# Predicciones (aquí no separamos train/test por el tamaño)
predictions = model.transform(data_ml)
print("\nPredicciones:")
display(predictions.select("features", "label", "probability", "prediction").toPandas())

# Evaluación del modelo
# AUC (Área bajo la curva ROC)

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction")
auc = evaluator.evaluate(predictions)
print(f"\nAUC del modelo (sobre el mismo conjunto de entrenamiento): {auc:.3f}")

# Calculo de precisión
accuracy = predictions.filter(col("label") == col("prediction")).count() / predictions.count()
print(f"Precisión (accuracy) en este conjunto: {accuracy:.2f}")


Dataset para MLlib listo:


,features,label
0,"[8900.0, 62000.0, 5.2, 3.8, 8.5, 12.3]",1
1,"[2500.0, 45000.0, 15.4, 4.5, 6.2, 14.1]",1
2,"[6000.0, 15000.0, 14.8, 10.1, 9.3, 11.0]",1
3,"[2200.0, 43000.0, 7.2, 1.8, 7.8, 13.0]",0
4,"[10000.0, 34000.0, 9.5, 13.2, 11.5, 13.8]",1



Predicciones:


,features,label,probability,prediction
0,"[8900.0, 62000.0, 5.2, 3.8, 8.5, 12.3]",1,"[1.3490343117437522e-08, 0.9999999865096569]",1.0
1,"[2500.0, 45000.0, 15.4, 4.5, 6.2, 14.1]",1,"[1.5930751148184658e-08, 0.9999999840692488]",1.0
2,"[6000.0, 15000.0, 14.8, 10.1, 9.3, 11.0]",1,"[4.728396725102802e-09, 0.9999999952716033]",1.0
3,"[2200.0, 43000.0, 7.2, 1.8, 7.8, 13.0]",0,"[0.9999999743387815, 2.56612184834637e-08]",0.0
4,"[10000.0, 34000.0, 9.5, 13.2, 11.5, 13.8]",1,"[3.8081707381913976e-13, 0.9999999999996192]",1.0



AUC del modelo (sobre el mismo conjunto de entrenamiento): 1.000
Precisión (accuracy) en este conjunto: 1.00
